### Makes the sendWaterSpeedDir.xml SFMC scrit 

Inits the simulated current field variables. Exactly the same as when plotting the glider data on the map, as these all need the same current field for plotting and SFMC script

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import glob as glob
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.crs as ccrs
# import cool_maps.plot as cplt
import cmocean.cm as cmo
import logging 
import bitstring
import time

start_lon = -86.5
end_lon = -85
filename = 'new_sendWaterSpeedDir.xml'
wheresave = '../SFMC_script/'
complete_filename = os.path.join(wheresave,filename)

def decimal_degrees_to_degrees_minutes(start_lon,end_lon):
    range_str = []
    unique_strings = []
    lon_map = []

    range_run = np.arange(start_lon, end_lon, 0.01) # range of numbers with 0.01 steps

    for i in range_run:
        deg = int(i)
        decimal_part = abs(i - deg)
        min = int(decimal_part * 60)
        # Formatting minutes to have leading zero if necessary
        min_str = f'{min:02d}'  # This ensures minutes are represented with leading zero if < 10
        string = f'{deg}{min_str}'

        # str = f'{deg}{min}'
        range_str.append(string)

    for string in range_str:
        if string not in unique_strings:
            unique_strings.append(string)
    
    # longitude strings to integers for mapping later
    for lon_str in unique_strings:
        londegstr = lon_str[1:3]  # Extracting degrees part without the negative sign
        lonminstr = lon_str[3:]   # Extracting minutes part for negative longitude
        lon_maploop = - (int(londegstr) + (int(lonminstr) / 60))

        lon_map.append(lon_maploop)

    return unique_strings,lon_map

# sections, 20, 20, 20, 15, 16
sec0 = 20
sec1 = 20
sec2 = 20
sec3 = 15
sec4 = 16

pltsec0 = 0
pltsec1 = sec0+sec1-1
pltsec2 = sec0+sec1+sec2-1
pltsec3 = sec0+sec1+sec2+sec3-1
pltsec4 = sec0+sec1+sec2+sec3+sec4-1

west0 = [45]*sec0
west1 = [30]*sec1
west2 = [20]*sec2
west3 = [0]*sec3
west4 = [340]*sec4

water_dir_deg = west0+west1+west2+west3+west4

rads = []
for i in water_dir_deg:
    rads_loop = math.radians(i)
    rads.append(rads_loop)

speed0 = [0.6]*sec0
speed1 = [0.4]*sec1
speed2 = [0.3]*sec2
speed3 = [0.1]*sec3
speed4 = [0.3]*sec4

water_speed = speed0 + speed1 + speed2 + speed3 + speed4

lon_range_str,lon_map = decimal_degrees_to_degrees_minutes(start_lon,end_lon)
loopnum = len(lon_range_str)
lat_map = [20.55]*len(lon_map)

print(f'Length of things should be equal: {len(lon_map)} = {len(lon_range_str)} = {len(rads)} = {len(water_speed)} = {len(lat_map)}')

Length of things should be equal: 91 = 91 = 91 = 91 = 91


### Send two commands, s_water_direction and s_water_speed

In [ ]:
# may have to hit run a few times because it does it so fast it breaks

header = (
    '<!-- From checkNoComms. This sends updated current direction before sendResume. This state "simulates" a current field\n'
    'by matching the longitude from surface dialogs, then !put s_water_direction and s_water_speed to what it should be for\n'
    'the location of the glider before diving. It only reads the logitude part of the surface dialog, from following part\n'
    'in quotes: GPS Location:  2101.563 N "-8629".289 E measured.\n'
    '-->\n'
    '<state name="sendWaterDirection">\n'
    ' 	<transitions>\n')

checkNoComms = (
    '    <!-- The start state of the script. -->\n'
    '    <initialState name="checkNoComms">\n'
    '        <transitions>\n'
    # '            <!-- start_when(enum) 9 says this at the top of surface. Send to to sendWaterSpeedDir-->\n'
    # '            <transition matchExpression="Because:timeout expired" toState="sendWaterSpeedDir">\n'
    # '            </transition>\n'
    # '            <!-- Check what glider surfaced for. If no_comms, send to to sendWaterSpeedDir"-->\n'
    # '            <transition matchExpression="Because:no comms for a while" toState="sendWaterSpeedDir">\n'
    # '            </transition>\n'
    # '            <!-- Happens during testing so put here. Send to to sendWaterSpeedDir -->\n'
    # '            <transition matchExpression="Because:nothing commanded" toState="sendWaterSpeedDir">\n'
    # '            </transition>\n'
    '            <!-- Must wait for glider to calculate water velocity! Send to to sendWaterSpeedDir -->\n'
    '            <transition matchExpression="Calculations COMPLETE" toState="sendWaterSpeedDir">\n'
    '            </transition>\n'
    '        </transitions>\n'
    '    </initialState>\n\n')

sendWaterSpeedDirHeader = (
    '<!-- From checkNoComms. This sends updated current direction before sendResume. This state "simulates" a current field\n'
    '    by matching the longitude from surface dialogs, then !put s_water_direction and s_water_speed to what it should be for\n'
    '    the location of the glider before diving. It only reads the logitude part of the surface dialog, from following part\n'
    '    in quotes: sensor:m_lon(lon)=\'-8630\'\n'
    '    -->\n'
    '    <state name="sendWaterSpeedDir">\n'
    '        <transitions>\n')

footer = (
    '<!-- SEND RESUME/DIVE ========================================= -->\n'
    '    <state name="sendResume">\n'
    '        <transitions>\n'
    '            <transition matchExpression="xxx command verify fail xxx" toState="sendResume">\n'
    '            </transition>\n'
    '            <!-- Ready to dive, but send report first! MUST SEND TWO CTLR-R\'S OR ELSE IT DOESN\'T WORK\n'
    '                 Script needs more time after the !put so sends two -->\n'
    '            <transition matchExpression="Hit Control-R to RESUME" toState="checkNoComms">\n'
    '                <action type="glider" command="Ctrl-R">\n'
    '                </action>\n'
    '                <action type="glider" command="Ctrl-R">\n'
    '                </action>\n'
    '            </transition>\n'
    '            <transition timeout="10" toState="checkNoComms">\n'
    '            </transition>\n'
    '        </transitions>\n'
    '    </state>\n'
    '\n'
    '    <finalState name="final">\n'
    '    </finalState>\n'
    '</gliderScript>\n')

file = open(complete_filename, 'w')
file.write(header)
for zz in range(0,loopnum):
    file.write(f' 		<transition matchExpression="{lon_range_str[zz]}" toState="sendResume">\n')
    file.write(f' 			<action type="glider" command="!put s_water_direction {rads[zz]}"> <!-- # in {np.around(water_dir_deg[zz])} -->\n')
    file.write(' 			</action>\n')
    file.write(f' 			<action type="glider" command="!put s_water_speed {water_speed[zz]}">\n')
    file.write(' 			</action>\n')
    file.write(' 		</transition>\n')
file.write(' 	</transitions>\n')
file.write(' </state>\n')
file.write('\n')
file.write('\n')
file.write('\n')

print(f'Processed: {filename} in {wheresave}')

### Send Four Commands (didn't work though, see manual)

In [6]:
# may have to hit run a few times because it does it so fast it breaks

header = (
    '<gliderScript>\n\n'
    )

file = open(complete_filename, 'w')
file.write(header)
file.write(checkNoComms)
file.write(sendWaterSpeedDirHeader)
for zz in range(0, loopnum):
    # file.write(f'           <transition matchExpression="   sensor:m_lon(lon)={lon_range_str[zz]}" toState="sendResume">\n')
    file.write(f'           <transition matchExpression="={lon_range_str[zz]}" toState="sendResume">\n')
    file.write(f'              <action type="glider" command="!put s_water_direction {rads[zz]}">\n') 
    file.write('              </action>\n')
    file.write(f'              <action type="glider" command="!put s_water_speed {water_speed[zz]}">\n')
    file.write('              </action>\n')
    file.write(f'              <action type="glider" command="!put X_PRIOR_SEG_WATER_VX 0">\n')
    file.write('              </action>\n')
    file.write(f'              <action type="glider" command="!put X_PRIOR_SEG_WATER_VY 0">\n')
    file.write('              </action>\n')
    file.write('            </transition>\n')
file.write('       </transitions>\n')
file.write('   </state>\n')
file.write('\n')
file.write(footer)


880